# AG3- Actividad Guiada 3

Ana Grau de la Herrán

URL: https://github.com/AnaGrauHerran/03MAIR-Algoritmos-de-Optimizacion/tree/master/AG3

In [1]:
#Decorador para calcular_tiempo a la función para la cual queremos calcular el tiempo de ejecución(Isabel Vázquez)

from time import time
#Función para calcular el tiempo de ejecución
def calcular_tiempo(f):
   
    def wrapper(*args, **kwargs):       
        inicio = time()       
        resultado = f(*args, **kwargs)       
        tiempo = time() - inicio
        print("Tiempo de ejecución para algoritmo: "+str(tiempo))
        return resultado
    
    return wrapper

In [2]:
# Descargar los datos

import urllib.request

file = "swiss42.tsp"

urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/swiss42.tsp", file)

('swiss42.tsp', <http.client.HTTPMessage at 0x15039739d68>)

In [3]:
# Instalar y cargar modulos
!pip install tsplib95

twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [4]:
import tsplib95
import random
from math import e

# cargar los datos del problema descargado
datos_problema = tsplib95.load_problem(file)

# los nodos se pueden obtener así
nodos_problema = list(datos_problema.get_nodes())

# las aristas se obtienen del problema así
aristas_problema = list(datos_problema.get_edges())

In [5]:
#############
# Mejora 1
#############
# El método factorial no entra en bucle infinito con datos equivocados

# Devuelve el factorial de un numero
def factorial(n):
    if n==0 or n==1:
        return 1
    elif n < 0:
        print("Solo podemos calcular factoriales de números positivos")
    else:
        return n * factorial(n-1)
    
# Se genera una solucion aleatoria con comienzo en 0
def crear_solucion(nodos):
    solucion = [0]
    size = len(nodos)
    for i in range(size - 1):
        solucion = solucion + [random.choice(list(set(nodos) - set({0}) - set(solucion)))]
    return solucion

# Devuelve la distancia entre dos nodos
def distancia(a, b, problema):
    return problema.wfunc(a,b)

# Devuelve la distancia total de una trayectoria
def distancia_total(solucion, problema):
    distancia_total = 0
    size = len(solucion)
    for i in range(size - 1):
        distancia_total += distancia(solucion[i], solucion[i+1], problema)
    return distancia_total + distancia(solucion[size - 1], solucion[0], problema)

In [6]:
# Comprobación que los métodos descritos compilan
solucion = crear_solucion(nodos_problema)
distancia(0,1, datos_problema)
distancia_total(solucion, datos_problema)

4699

In [7]:
#############
# Mejora 2
#############
# Corregir errata en la busqueda aleatoria pues imprimia la ultima solucion encontrada
# en vez de la mejor_solucion

import sys

# Dado un problema y un número n de iteraciones
# Busca n soluciones aleatorias y devuelve
# la mejor de las n posibles soluciones
@calcular_tiempo
def busqueda_aleatoria(problema, n):
    # problema = datos del problema
    # n = numero de iteraciones
    
    #Nodos
    nodos = list(problema.get_nodes())
    
    mejor_solucion = []
    mejor_distancia = sys.maxsize
    
    # para cada iteracion
    for i in range(n):
        # se genera solucion aleatoria
        solucion = crear_solucion(nodos)

        # se comprueba si es mejor que las anteriores
        distancia_solucion = distancia_total(solucion, problema)
        if distancia_solucion < mejor_distancia:
            mejor_solucion = solucion
            mejor_distancia = distancia_solucion
      
    print("La mejor solucion encontrada es ", end = "")
    print(mejor_solucion)
    print("con una distancia total de ", end="")
    print(mejor_distancia)
    return mejor_solucion

print(busqueda_aleatoria(datos_problema, 100))        

La mejor solucion encontrada es [0, 13, 19, 6, 20, 30, 5, 1, 9, 40, 41, 25, 11, 27, 37, 16, 38, 33, 28, 24, 7, 26, 39, 8, 23, 4, 15, 3, 35, 2, 36, 31, 14, 32, 10, 29, 34, 17, 12, 21, 22, 18]
con una distancia total de 4140
Tiempo de ejecución para algoritmo: 0.0379793643951416
[0, 13, 19, 6, 20, 30, 5, 1, 9, 40, 41, 25, 11, 27, 37, 16, 38, 33, 28, 24, 7, 26, 39, 8, 23, 4, 15, 3, 35, 2, 36, 31, 14, 32, 10, 29, 34, 17, 12, 21, 22, 18]


In [8]:
# Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos)
# Si hay N nodos se generan (N-1)x(N-2)/2 soluciones
# Este metodo tiene un numero de iteraciones fija (dependiente del numero de nodos)
# En vez de buscar soluciones aleatorias, busca soluciones generando vecinos (intercambiando puntos)
def genera_vecina(solucion, problema):   
    # Inicializar variables para solucion
    mejor_solucion = []
    mejor_distancia = sys.maxsize

    size = len(solucion)
    # para todo valor (menos el último)
    for i in range(1, size - 1):
        # se genera solucion intercambiando con los siguientes
        for j in range(i+1, size):
            # solucion [1-n] => vecina = 1,...,i - 1, j , i +1,...., j-1, i, j+1,....,n
            vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]
            # se comprueba si la solucion vecina mejora la solucion actual
            distancia_vecina = distancia_total(vecina, problema)
            if distancia_vecina <= mejor_distancia:
                mejor_distancia = distancia_vecina
                mejor_solucion = vecina

    return mejor_solucion

#############
# Mejora 3
#############
# Crear generador de vecinos que escoja sublista e invierta el orden del resto de elementos
# metodo usado con este generador abajo
def genera_vecina_invirtiendo(solucion, problema):   
    # Inicializar variables para solucion
    mejor_solucion = []
    mejor_distancia = sys.maxsize

    size = len(solucion)
    # para todo valor (menos el último)
    for i in range(1, size - 1):
        # se genera solucion intercambiando con los siguientes
        for j in range(i+1, size):
            # solucion [1-n] => vecina = 1,...,i -1 , j, j-1, ... , i, j+1,..., n
            vecina = solucion[:i] + solucion[j:i-1:-1] + solucion[j+1:]
            # se comprueba si la solucion vecina mejora la solucion actual
            distancia_vecina = distancia_total(vecina, problema)
            if distancia_vecina <= mejor_distancia:
                mejor_distancia = distancia_vecina
                mejor_solucion = vecina

    return mejor_solucion

#############
# Mejora 4
#############
# Crear generador de vecinos que escoja sublista y baraje el resto de elementos
# metodo usado con este generador abajo
import random
def genera_vecina_barajando(solucion, problema):   
    # Inicializar variables para solucion
    mejor_solucion = []
    mejor_distancia = sys.maxsize

    size = len(solucion)
    # para todo valor (menos el último)
    for i in range(1, size - 1):
        # se genera solucion intercambiando con los siguientes
        for j in range(i+1, size):
            # solucion [1-n] => vecina = 1,...,i -1 , j-1, ... ,i, j-4, j,...,j+1, ..., n
            copia = solucion[i:j]
            random.shuffle(copia)
            vecina = solucion[:i] + copia + solucion[j:]
            # se comprueba si la solucion vecina mejora la solucion actual
            distancia_vecina = distancia_total(vecina, problema)
            if distancia_vecina <= mejor_distancia:
                mejor_distancia = distancia_vecina
                mejor_solucion = vecina

    return mejor_solucion

In [9]:
@calcular_tiempo
def busqueda_local(problema, n, generador):
    # problem = datos del problema
    # n = numero de iteraciones
    
    mejor_solucion = []
    mejor_distancia = sys.maxsize
    
    nodos = list(problema.get_nodes())
    solucion_referencia = crear_solucion(nodos)
    
    for i in range(n):
        # seleccionar solucion vecina con generador seleccionado
        vecina = generador(solucion_referencia, problema)
        # se comprueba si la solucion vecina mejora la solucion actual
        distancia_vecina = distancia_total(vecina, problema)
        if distancia_vecina <= mejor_distancia:
            mejor_solucion = vecina
            mejor_distancia = distancia_vecina
            
        solucion_referencia = vecina
        
    print("La mejor solucion encontrada es ", end="")
    print(mejor_solucion)
    print("con una distancia total de ", end="")
    print(mejor_distancia)
    return mejor_solucion

print("busqueda local intercambiando elementos")
busqueda_local(datos_problema, 100, genera_vecina)
print("------------------------------------------------")
print("busqueda local invirtiendo sublista de elementos")
busqueda_local(datos_problema, 100, genera_vecina_invirtiendo)
print("------------------------------------------------")
print("busqueda local barajando sublista de elementos")
print(busqueda_local(datos_problema, 100, genera_vecina_barajando))

busqueda local intercambiando elementos
La mejor solucion encontrada es [0, 34, 33, 20, 17, 15, 37, 36, 35, 31, 32, 27, 2, 3, 4, 6, 5, 19, 13, 18, 12, 11, 25, 41, 23, 9, 21, 40, 24, 39, 22, 38, 30, 7, 14, 16, 26, 10, 8, 29, 28, 1]
con una distancia total de 1704
Tiempo de ejecución para algoritmo: 5.5448222160339355
------------------------------------------------
busqueda local invirtiendo sublista de elementos
La mejor solucion encontrada es [0, 32, 31, 17, 36, 35, 20, 33, 34, 38, 22, 24, 40, 21, 39, 9, 23, 41, 25, 11, 12, 10, 8, 29, 30, 28, 27, 2, 3, 4, 6, 5, 26, 18, 13, 19, 14, 16, 15, 37, 7, 1]
con una distancia total de 1362
Tiempo de ejecución para algoritmo: 5.3449554443359375
------------------------------------------------
busqueda local barajando sublista de elementos
La mejor solucion encontrada es [0, 16, 37, 17, 35, 30, 21, 41, 4, 3, 1, 6, 14, 19, 5, 26, 10, 22, 38, 33, 20, 31, 36, 15, 32, 28, 29, 8, 9, 39, 24, 40, 23, 25, 11, 12, 18, 13, 7, 2, 27, 34]
con una distancia t

In [10]:
# Generador de 1 solucion vecina intercambiando
# dos indices aleatorios
# La variable problema no es necesaria, pero se añade para
# igualar la estructura de los otros generadores
def genera_vecina_aleatorio(solucion, problema):
    size = len(solucion)
    # seleccion de aleatoria de primer indice
    i = random.choice(range(1, size))
    # eliminar del listado de indices el indice que ya ha salido
    # para que no salga repetido
    j = random.choice(list(set(range(1, size)) - {i}))
    # generar vecino intercambiando los indices
    vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]
    return vecina

def probabilidad(temperatura, error_distancia):
    r = random.random()
    return r <= (e**(-1*error_distancia)/(temperatura*1.0))

def bajar_temperatura(temperatura):
    return temperatura - 1

In [11]:
#############
# Mejora 5
#############
# Adaptar el problema para aceptar diferente generadores de vecinos
# incluyendo los definidos arriba
@calcular_tiempo
def recocido_simulado(problema, temperatura, generador):
    # problema = datos del problema
    # temperatura = temperatura
    
    nodos = list(problema.get_nodes())
    solucion_referencia = crear_solucion(nodos)
    distancia_referencia = distancia_total(solucion_referencia, problema)
    
    mejor_solucion = []
    mejor_distancia = sys.maxsize
    
    while temperatura >0:
        # Genera una solucion vecina(aleatoria)
        vecina = generador(solucion_referencia,  problema)
        
        # Calcula su valor(distancia)
        distancia_vecina = distancia_total(vecina, problema)
        
        # Si es la mejor solucion de todas se guarda
        if distancia_vecina < mejor_distancia:
            mejor_solucion = vecina
            mejor_distancia = distancia_vecina
            
        # Si la nueva vecina es mejor se cambia y si es peor
        # se cambia según una probablidad dependiente de temperatura y de
        # |distancia_referencia - distancia_vecina|
        if distancia_vecina < distancia_referencia or probabilidad(temperatura, abs(distancia_referencia -distancia_vecina)):
            solucion_referencia = vecina
            distancia_referencia = distancia_vecina
            
        temperatura = bajar_temperatura(temperatura)
        
    print("La mejor solucion encontrada es ", end="")
    print(mejor_solucion)
    print("con una distancia total de ", end="")
    print(mejor_distancia)
    return mejor_solucion

print("busqueda local intercambiando elementos aleatorios")
recocido_simulado(datos_problema, 100, genera_vecina_aleatorio)
print("------------------------------------------------")
print("busqueda local intercambiando elementos contiguos")
recocido_simulado(datos_problema, 100, genera_vecina)
print("------------------------------------------------")
print("busqueda local invirtiendo sublista de elementos")
recocido_simulado(datos_problema, 100, genera_vecina_invirtiendo)
print("------------------------------------------------")
print("busqueda local barajando sublista de elementos")
print(recocido_simulado(datos_problema, 100, genera_vecina_barajando))

busqueda local intercambiando elementos aleatorios
La mejor solucion encontrada es [0, 7, 17, 16, 3, 39, 24, 36, 37, 14, 32, 29, 34, 20, 21, 40, 1, 6, 19, 4, 8, 22, 38, 27, 2, 25, 10, 13, 15, 26, 9, 5, 11, 28, 35, 33, 30, 41, 23, 12, 18, 31]
con una distancia total de 3528
Tiempo de ejecución para algoritmo: 0.008997678756713867
------------------------------------------------
busqueda local intercambiando elementos contiguos
La mejor solucion encontrada es [0, 22, 38, 7, 14, 16, 19, 41, 23, 40, 24, 21, 39, 2, 3, 1, 6, 5, 26, 18, 12, 25, 11, 13, 15, 37, 17, 31, 36, 35, 32, 28, 30, 29, 9, 8, 10, 4, 27, 34, 33, 20]
con una distancia total de 2021
Tiempo de ejecución para algoritmo: 5.632772207260132
------------------------------------------------
busqueda local invirtiendo sublista de elementos
La mejor solucion encontrada es [0, 1, 3, 27, 2, 4, 26, 18, 12, 11, 13, 19, 5, 6, 7, 14, 16, 15, 37, 17, 31, 36, 35, 20, 33, 34, 38, 22, 39, 21, 24, 40, 23, 41, 25, 10, 8, 9, 29, 30, 28, 32]
con 

In [12]:
def add_nodo_aleatorio(problema, hormiga, feromonas):
    # Establecer una funcion de probabilidad para
    # añadir un nuevo nodo dependiendo de los nodos
    # mas cercanos y de las feromonas depositadas
    nodos = list(problema.get_nodes())
    size = len(nodos)
    return random.choice(list(set(range(1, size)) - set(hormiga))) # elegir elemento aleatorio de una lista

#############
# Mejora 6
#############
# Flexibilizacion en la constante de uso para incremento
def incrementa_feromona(problema, feromonas, hormiga, constante):
    #Incrementar segun la calidad de la solucion
    #Añadir una cantidad inversamente proporcional a la distancia total
    for i in range(len(hormiga) -1):
        feromonas[hormiga[i]][hormiga[i+1]] += constante/distancia_total(hormiga, problema)
    return feromonas

def evaporar_feromonas(feromonas, problema):
    # Evapora 0.3 el valor de la feromona, sin que baje de 1
    nodos = list(problema.get_nodes())
    size = len(nodos)
    feromonas = [[ max(feromonas[i][j] - 0.3, 1) for i in range(size)] for j in range(size)]
    return feromonas

#############
# Mejora 7
#############
# Combinar evaporar e incrementa en una funcion de actualizacion de feromonas
# para simplificar modificaciones en la actualizacion
def actualizar_feromonas(feromonas, problema, camino_hormiga):
    # Incrementa feromonas en esa arista (constante = 1000)
    feromonas = incrementa_feromona(problema, feromonas, camino_hormiga, 1000)
    # Evaporar feromonas
    return evaporar_feromonas(feromonas, problema)

In [13]:
#############
# Mejora 8
#############
# Mejorar la funcion add_nodo usando la formula de probabilidad
# para la seleccion de aristas
import numpy as np

# Funcion a mejorar usando fórmula
def add_nodo_probabilidad(problema, camino_hormiga, feromonas):
    # Establecer una funcion de probabilidad para
    # añadir un nuevo nodo dependiendo de los nodos
    # mas cercanos y de las feromonas depositadas
    nodos = list(problema.get_nodes())
    size = len(nodos)
    
    i = len(camino_hormiga) # calculamos las probabilidades para el nodo i que es el siguiente en el camino
    nodos_en_camino = set(camino_hormiga)
    # Calculo de las distancias de los nodos Vij^beta
    distancias = []
    nodo_i = nodos[i]
    for j in range(size):
        nodo_j = nodos[j]
        if nodo_j not in nodos_en_camino: # solo trabajamos con los nodos que no han sido seleccionados
            distancias.append((1 + distancia(nodo_i, nodo_j, problema)**(beta)))#añadimos una constante para evitar ceros en el denominador
        else:
            distancias.append(0)# debemos añadir valor para que el tamaño de distancias todavia permita mapear nodo e indice
    # Calculo peso = sum_l(Til^alpha * Vil^beta)
    peso = 0
    for j in range(size):
        if nodos[j] not in nodos_en_camino:
            peso += (feromonas[i][j] ** alpha) / (distancias[j])

    # calculo probabilidades pij = Tij^alpha/ (peso * distancias[j]) = Tij^alpha * Vij^beta / peso 
    #                            = Tij^alpha * Vij^beta / sum_l(Til^alpha * Vil^beta)
    nodos_restantes = []
    probabilidad_nodos = []
    for j in range(size):
        nodo_j = nodos[j]
        if nodo_j not in nodos_en_camino:
            nodos_restantes.append(nodo_j)
            probabilidad_nodos.append((feromonas[i][j] ** alpha)/(peso * distancias[j]))

    return np.random.choice(nodos_restantes, p=probabilidad_nodos)

In [14]:
#############
# Mejora 8
#############
# Modificar la funcion evaporar_feromonas usando producto (ver https://es.wikipedia.org/wiki/Algoritmo_de_la_colonia_de_hormigas)
# con parametros coeficiente y constante variables
# Tij => (1-r)Tij+ sum_k Dij^k [evaporar + incrementar]
# r = coeficiente de evaporacion
# incremento de feromona ya es como en la formula
# Dij^k = Q/Lk si la hormiga k usa el camino ij
# Q una constante (en nuestro caso Q = 1000)
# Lk el costo (distancia) de la ruta de la hormiga k
def evaporar_feromonas_mejorada(feromonas, problema):
    # Evapora 0.3 el valor de la feromona, sin que baje de 1
    nodos = list(problema.get_nodes())
    size = len(nodos)
    feromonas = [[ coeficiente * feromonas[i][j] for i in range(size)] for j in range(size)]
    return feromonas

def actualizar_feromonas_mejorada(feromonas, problema, camino_hormiga):
    # Incrementa feromonas en esa arista
    feromonas = evaporar_feromonas_mejorada(feromonas, problema)
    # Evaporar feromonas
    return incrementa_feromona(problema, feromonas, camino_hormiga, constante)

In [15]:
@calcular_tiempo
def hormigas(problema, n, add_nodo, actualizar):
    # problema = datos del problema
    # n = numero de agentes(hormigas)
    
    # Nodos
    nodos = list(problema.get_nodes())
    size = len(nodos)
    # Aristas
    aristas = list(problema.get_edges())
    
    # Inicializa las aristas con una cantidad inicial de feromonas:1
    feromonas = [[1 for _ in range(size)] for _ in range(size)]
    
    # Se generan los agentes(hormigas) que seran estructuras de caminos desde 0
    hormigas = [[0] for _ in range(n)]
    
    # Recorre cada agente construyendo su "solucion"
    for hormiga in range(n):
        # Para cada agente se construye un camino
        for i in range(size - 1):
            # Elige el siguiente nodo
            nuevo_nodo = add_nodo(problema, hormigas[hormiga], feromonas)
            hormigas[hormiga].append(nuevo_nodo)
    
        # actualizar feromonas (incrementa y evapora)
        feromonas = actualizar(feromonas, problema, hormigas[hormiga])
        
    # Seleccionamos el mejor agente
    mejor_solucion = []
    mejor_distancia = sys.maxsize
    for hormiga in range(n):
        distancia_actual = distancia_total(hormigas[hormiga], problema)
        if distancia_actual < mejor_distancia:
            mejor_solucion = hormigas[hormiga]
            mejor_distancia = distancia_actual

    print("La mejor solucion encontrada es ", end="")
    print(mejor_solucion)
    print("con una distancia total de ", end="")
    print(mejor_distancia)


In [16]:
print("hormiga con add_nodo_aleatorio")
hormigas(datos_problema, 1000, add_nodo_aleatorio, actualizar_feromonas)
alpha = 0
for alpha_addition in range(5):
    beta = 1
    for beta_addition in range(5):
        print("----------------------------------------------------------------")
        print("hormiga con add_nodo_probabilidad, alpha:" + str(alpha) + " beta: " + str(beta))
        hormigas(datos_problema, 1000, add_nodo_probabilidad, actualizar_feromonas)
        beta += 0.5
    alpha += 0.5

hormiga con add_nodo_aleatorio
La mejor solucion encontrada es [0, 35, 20, 16, 13, 12, 5, 10, 41, 4, 23, 26, 37, 32, 29, 2, 39, 27, 40, 21, 34, 33, 22, 9, 8, 25, 28, 1, 17, 11, 7, 18, 15, 14, 19, 36, 6, 3, 24, 38, 30, 31]
con una distancia total de 3755
Tiempo de ejecución para algoritmo: 3.403068780899048
----------------------------------------------------------------
hormiga con add_nodo_probabilidad, alpha:0 beta: 1
La mejor solucion encontrada es [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41]
con una distancia total de 2834
Tiempo de ejecución para algoritmo: 9.372612476348877
----------------------------------------------------------------
hormiga con add_nodo_probabilidad, alpha:0 beta: 1.5
La mejor solucion encontrada es [0, 20, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 26, 21, 22, 23, 41, 25, 1, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39

La mejor solucion encontrada es [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 39, 23, 24, 25, 26, 27, 30, 29, 28, 31, 32, 33, 34, 35, 36, 37, 38, 40, 21, 41]
con una distancia total de 2760
Tiempo de ejecución para algoritmo: 9.467575311660767
----------------------------------------------------------------
hormiga con add_nodo_probabilidad, alpha:2.0 beta: 1.5
La mejor solucion encontrada es [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 41, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 24]
con una distancia total de 2673
Tiempo de ejecución para algoritmo: 9.014854192733765
----------------------------------------------------------------
hormiga con add_nodo_probabilidad, alpha:2.0 beta: 2.0
La mejor solucion encontrada es [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 41, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 24]
con una distanc

In [17]:
coeficiente = 0.1
for i in range(9):
    constante = 500
    for j in range(3):
        print("----------------------------------------------------------------")
        print("hormiga con actualiza_feromona_mejorada, coeficiente:" + str(coeficiente) + " constante: " + str(constante))
        hormigas(datos_problema, 1000, add_nodo_aleatorio, actualizar_feromonas_mejorada)
        constante += 500
    coeficiente += 0.1

----------------------------------------------------------------
hormiga con actualiza_feromona_mejorada, coeficiente:0.1 constante: 500
La mejor solucion encontrada es [0, 29, 21, 25, 11, 24, 30, 38, 4, 13, 37, 9, 10, 39, 28, 33, 17, 12, 14, 19, 15, 8, 5, 27, 20, 36, 35, 31, 6, 16, 18, 26, 1, 32, 34, 7, 22, 2, 3, 23, 40, 41]
con una distancia total de 3787
Tiempo de ejecución para algoritmo: 3.017270803451538
----------------------------------------------------------------
hormiga con actualiza_feromona_mejorada, coeficiente:0.1 constante: 1000
La mejor solucion encontrada es [0, 17, 33, 16, 15, 35, 37, 7, 34, 20, 5, 32, 38, 9, 22, 1, 10, 3, 14, 19, 26, 41, 24, 39, 21, 40, 28, 11, 27, 29, 36, 12, 6, 13, 2, 31, 18, 25, 8, 30, 23, 4]
con una distancia total de 3646
Tiempo de ejecución para algoritmo: 2.990283966064453
----------------------------------------------------------------
hormiga con actualiza_feromona_mejorada, coeficiente:0.1 constante: 1500
La mejor solucion encontrada es [

La mejor solucion encontrada es [0, 19, 11, 10, 18, 38, 25, 6, 15, 20, 32, 5, 26, 27, 33, 41, 24, 4, 8, 29, 28, 3, 31, 12, 2, 13, 30, 35, 23, 21, 9, 40, 39, 22, 34, 17, 37, 7, 36, 14, 1, 16]
con una distancia total de 3808
Tiempo de ejecución para algoritmo: 2.927340269088745
----------------------------------------------------------------
hormiga con actualiza_feromona_mejorada, coeficiente:0.7999999999999999 constante: 500
La mejor solucion encontrada es [0, 39, 2, 15, 26, 6, 31, 35, 17, 37, 14, 32, 12, 11, 3, 29, 16, 13, 36, 10, 8, 41, 18, 21, 24, 33, 5, 25, 28, 27, 38, 9, 22, 23, 4, 19, 20, 34, 1, 30, 40, 7]
con una distancia total de 3920
Tiempo de ejecución para algoritmo: 3.1232047080993652
----------------------------------------------------------------
hormiga con actualiza_feromona_mejorada, coeficiente:0.7999999999999999 constante: 1000
La mejor solucion encontrada es [0, 36, 31, 7, 19, 37, 1, 8, 40, 20, 3, 17, 34, 2, 21, 23, 29, 32, 35, 6, 33, 16, 18, 10, 11, 26, 13, 24, 30